Import packages

In [4]:
import ee
import pandas as pd
import os, glob, sys
import json
from datetime import datetime, timedelta, date
import matplotlib.pyplot as plt
import logging
logging.basicConfig(level=logging.INFO)

Authenticate and initialize GEE login 

(If you get 'oauth2client' error, try installing an older version of the package e.g. pip install oauth2client==3.0.0)


In [6]:
ee.Initialize()

## Functions
####################

In [7]:
def daterange(start_date, end_date):
    #function to loop through weeks
    for n in range(0,int ((end_date - start_date).days)):
        yield start_date + timedelta(n)
        
def getQABits(image, start, end, mascara):
    # Cloud mask for pixels with less than 20% cloud cover 
    # Compute the bits we need to extract.
    # Return a single band image of the extracted QA bits, giving the band a new name.
    pattern = 0
    for i in range(start,end-1):
        pattern += 2**i
    return image.select([0], [mascara]).bitwiseAnd(pattern).rightShift(start)
        
def maskS2clouds(image):
    #Return the masked and scaled Sentinal2 data, without the QA bands
    #Bits 10 and 11 are clouds and cirrus, respectively
    qa = image.select('QA60')
    cloudBitMask = getQABits(qa,10,10,'cloud')
    cirrusBitMask = getQABits(qa,11,11,'cirrus')
    return image.updateMask(cloudBitMask.eq(0).updateMask(cirrusBitMask.eq(0))).divide(10000).select("B.*").float().copyProperties(image, ["system:time_start", "system:index"])

In [8]:
def maskL8clouds(image):
    #Return the masked and scaled Landsat8 data, without the QA bands
    # Bits 3 and 5 are cloud shadow and cloud, respectively
    qa = image.select('pixel_qa')
    cloudShadowBitMask = getQABits(qa,3,3,'cloud_shadow')
    cloudBitMask = getQABits(qa,5,5,'cloud')
    cirrusBitMask = getQABits(qa,9,9,'cirrus')
    return image.updateMask(cloudShadowBitMask.eq(0)).updateMask(cloudBitMask.eq(0).updateMask(cirrusBitMask.eq(0))).divide(10000).select("B[0-9]*").float().copyProperties(image, ["system:time_start", "system:index"])

In [9]:
def maskL57clouds(image):
    # Return the masked image, scaled to reflectance, without the QA bands
    # Bits 1,2 and 3 are cloud shadow and cloud, respectively
    qa = image.select('sr_cloud_qa')
    cloudShadowBitMask = getQABits(qa,2,2,'cloud_shadow')
    cloudBitMask = getQABits(qa,1,1,'cloud')
    cirrusBitMask = getQABits(qa,3,3,'cirrus')
    return image.updateMask(cloudShadowBitMask.eq(0)).updateMask(cloudBitMask.eq(0).updateMask(cirrusBitMask.eq(0))).divide(10000).select("B[0-9]*").float().copyProperties(image, ["system:time_start", "system:index"])

In [10]:
def addNDVI_l8(image):
    # compute NDVI for Landsat 8
    ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

In [11]:
def addNDVI_s2(image):
    # compute NDVI for Sentinal 2
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

In [12]:
def addNDVI_l57(image):
    # compute NDVI for Landsat 5-7
    ndvi = image.normalizedDifference(['B4', 'B3']).rename('NDVI')
    return image.addBands(ndvi)

In [13]:
def _reduce_region(image):
    # Spatial aggregation function for a single image and a polygon feature
    # FEature needs to be rebuilt because the backend doesn't accept to map
    # functions that return dictionaries
    size = 30
    fnc = ee.Reducer.mean()
    stat_dict = image.reduceRegion(fnc, geom, size);
    return ee.Feature(None, stat_dict)

In [14]:
def simplify(fc):
    def feature2dict(f):
        '''
        converting feature to dict user might have to modify identifires
        to correctly link to specified image collection - its hard codded in this 
        case for chirps precip and L7/L8 datasets 
        '''
        
        tmp = None
        if 'precip' in list(fc['features'][0]['properties'].keys())[0]:
            tmp = f['id']
        if f['id'][:2] == '1_':
            tmp = f['id'].split('_')[3]
        if f['id'][:2] == '2_':
            tmp = f['id'][2:10]
        if f['id'][0] =='L' or f['id'][:2] == '20':
            if 'LC' in f['id'] or 'LE' in f['id']:
                tmp = f['id'].split('_')[2]
            if 'T3' in f['id']:
                tmp = f['id'].split('_')[0]
                tmp = tmp[:8]
        id = tmp
        out = f['properties']
        out.update(id=id)
        return out
    out = [feature2dict(x) for x in fc['features']]
    return out

In [36]:
def compute(fc,flag, duration = None):
    """ 
    function to perform spatial aggregation & extract image collection data
    inta a pandas dataframe 
    
    INPUTS:
    FC      : feature collection information
    Flag    : 1 - Precipitation
              0 - NDVI data
    duration: M - Monthly statistics
              Y - Yearly statistics
              
    For precipitation total sum is used whereas for NDVI monthly max is used by default
    """
    
    sfc = simplify(fc)
    a = pd.DataFrame.from_dict(sfc)
    a.index = pd.to_datetime(a['id'], infer_datetime_format = True)
    a = a.drop('id', axis = 1)
    
    if flag == 1:   # Precip
        if duration == 'M':
            a = a.resample('M').sum()
        if duration == 'Y':
            a = a.resample('Y').sum()   
    
    if flag == 0:   # NDVI
        if duration != None:
            if duration == 'M':
                a = a.resample('M').max()
            if duration == 'Y':
                a = a.resample('Y').max()
        else:
            print('Invalid parameters...exiting now')
            sys.exit()
    return a

In [23]:
input_dir = "/Users/vmishra2/Documents/SERVIR/Misc/Resileience/Niger/"
geojsons = glob.glob(input_dir+"2*.geojson")

In [37]:
for geojson in geojsons:
    fname = geojson.split('/')[-1].split('.')[0]
    
    with open(geojson,"r") as f:
        data = f.read()
        fc_str = json.loads(data)
        
        df_ch = pd.DataFrame()
        df_l8 = pd.DataFrame()
        df_l7 = pd.DataFrame()
        df_s2 = pd.DataFrame()
        
        df_l8['Date'] = pd.date_range('1/1/2002','12/31/2020')
        df_l7['Date'] = pd.date_range('1/1/2002','12/31/2020')
        df_s2['Date'] = pd.date_range('1/1/2002','12/31/2020')
        
        df_l8 = df_l8.set_index('Date')
        df_l7 = df_l7.set_index('Date')
        df_s2 = df_s2.set_index('Date')
        
        print(str(fname) +' has ', len(fc_str['features']),' features')
        for ii in range(len(fc_str['features'])):
            aoi_ = fc_str['features'][ii]['geometry']
            
            name1 = fc_str['features'][ii]['properties']['Name']
            name2 = fc_str['features'][ii]['properties']['FID']
            name = name1+'_'+str(name2)
            
            #get base collections and subset to aoi_
            l8_boa = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR").filterBounds(aoi_).filterDate('2012-1-1','2020-12-31')
            l7_boa = ee.ImageCollection("LANDSAT/LE07/C01/T1_SR").filterBounds(aoi_).filterDate('2002-1-1','2020-12-31')
            s2_boa = ee.ImageCollection("COPERNICUS/S2_SR").filterBounds(aoi_).filterDate('2012-1-1','2020-12-31')
            precp = ee.ImageCollection("UCSB-CHG/CHIRPS/PENTAD").filterBounds(aoi_).filterDate('2002-1-1','2020-12-31').select('precipitation')
            
            #maps bands to each other  
            bands_l8 = ['B1','B2','B3','B4','B5','B6','B7']   #var bands_l8 = ['B1','B2','B3','B4','B5','B6','B7','QA60'];
            bands_s2 = ['B1','B2','B3','B4','B8','B11','B12']   #var bands_s2 = ['B1','B2','B3','B4','B8','B11','B12','QA60'];
            
            # perform cloud masking
            s2_boa_cm = s2_boa.map(maskS2clouds)
            l8_boa_cm = l8_boa.map(maskL8clouds)
            l7_boa_cm = l7_boa.map(maskL57clouds)
            
            # get NDVI
            l8_ndvi = l8_boa_cm.map(addNDVI_l8).select('NDVI')
            l7_ndvi = l7_boa_cm.map(addNDVI_l57).select('NDVI')
            s2_ndvi = s2_boa_cm.map(addNDVI_s2).select('NDVI')
            
            geom = ee.Geometry.MultiPolygon(aoi_["coordinates"])
            size = 30
            fnc = ee.Reducer.mean()
            
            # CHIRPS Precip
            fc = precp.map(_reduce_region).getInfo()
            chirp = compute(fc,1,'M')
            chirp.columns = [name]

            # L8 NDVI
            fc = l8_ndvi.map(_reduce_region).getInfo()
            l8vi =  compute(fc,0,'M')
            l8vi.columns = [name]
            
            # L7 NDVI
            fc = l7_ndvi.map(_reduce_region).getInfo()
            l7vi =  compute(fc,0,'M')
            l7vi.columns = [name]
            
            # S2 NDVI
            fc = s2_ndvi.map(_reduce_region).getInfo()
            s2vi =  compute(fc,0,'M')
            s2vi.columns = [name]

            #Store extracted values to pandas dataframe
            df_l8 = df_l8.merge(l8vi, left_index=True, right_index=True, how='left')
            df_l7 = df_l7.merge(l7vi, left_index=True, right_index=True, how='left')
            df_s2 = df_s2.merge(s2vi, left_index=True, right_index=True, how='left')
            
            if ii == 0:
                df_ch = chirp
                df_l8 = l8vi
                df_l7 = l7vi
                df_s2 = s2vi
            else:
                df_ch = df_ch.merge(chirp, left_index=True, right_index=True, how='left')
                df_l8 = df_l8.merge(l8vi, left_index=True, right_index=True, how='left')
                df_l7 = df_l7.merge(l7vi, left_index=True, right_index=True, how='left')
                df_s2 = df_s2.merge(s2vi, left_index=True, right_index=True, how='left')
            sys.exit()
            
        # save data into csv files
        df_ch.to_csv(input_dir+yr+'_chirps.csv')
        df_l8.to_csv(input_dir+yr+'_L8.csv')
        df_l7.to_csv(input_dir+yr+'_L7.csv')
        df_s2.to_csv(input_dir+yr+'_S2.csv')
        
        
sys.exit()
            


2018 has  27  features


SystemExit: 

/Users/vmishra2/miniconda3/envs/ee/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3449: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [39]:
df_l8.head()

,Gochiro_0
id,
2013-04-30,0.005741
2013-05-31,0.136476
2013-06-30,0.162010
2013-07-31,0.161469
2013-08-31,0.248491
